<a href="https://colab.research.google.com/github/unknownguy24/Deep-Learning/blob/main/Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('train.csv',encoding='latin-1')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df= df.dropna()

In [ ]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [ ]:
## Get the Dependent features
y=df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.9.2'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

In [ ]:
#OneHot Representation

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])    #substituting everything except letters with a space
  review = review.lower()                                    #lowering
  review = review.split()                                    #splitting string to a list
    
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]  #applying porterstemmer and removing stop words
  review = ' '.join(review)                                  #joining all the words
  corpus.append(review)

In [ ]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]

In [ ]:
onehot_repr[0]

[3143, 4895, 1745, 1547, 55, 2207, 3751, 588, 927, 1752]

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  588  927 1752]
 [   0    0    0 ... 1081 2987 4241]
 [   0    0    0 ... 1661 3158 3964]
 ...
 [   0    0    0 ... 3899 4796 2023]
 [   0    0    0 ... 1298 3215 2916]
 [   0    0    0 ...  151 3152 3870]]


In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 10ms/step - loss: 0.3364 - accuracy: 0.8379 - val_loss: 0.1863 - val_accuracy: 0.9190
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1331 - accuracy: 0.9480 - val_loss: 0.1885 - val_accuracy: 0.9218
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0877 - accuracy: 0.9674 - val_loss: 0.2154 - val_accuracy: 0.9221
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0530 - accuracy: 0.9802 - val_loss: 0.2713 - val_accuracy: 0.9163
Epoch 5/10
192/192 [==============================] - 1s 8ms/step - loss: 0.0345 - accuracy: 0.9876 - val_loss: 0.3489 - val_accuracy: 0.9107
Epoch 6/10
192/192 [==============================] - 1s 8ms/step - loss: 0.0184 - accuracy: 0.9941 - val_loss: 0.3955 - val_accuracy: 0.9145
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0115 - accuracy: 0.9959 - val_loss: 0.3930 - val_accuracy: 0.9100
Epoch

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [118]:
predictions = (model.predict(X_test) > 0.5)*1

189/189 [==============================] - 1s 4ms/step


In [119]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.9149958574979288

In [120]:
from sklearn.metrics import confusion_matrix

In [121]:

confusion_matrix(y_test,predictions)

array([[3125,  294],
       [ 219, 2397]])